<a href="https://colab.research.google.com/github/YolandaMDavis/wildtrack-iqa/blob/task-amenability/task_amenability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

# get downstream task model
shutil.copy(PARENT_DIR + '/task-amenability/model.pt', 'model.pt')


Mounted at /content/drive


'model.pt'

In [2]:
# clone the repository and move data file
!git clone https://github.com/YolandaMDavis/wildtrack-iqa.git
!mv WildTrack_Raw.zip wildtrack-iqa/.
!mv model.pt wildtrack-iqa/.
%cd "wildtrack-iqa"
!git checkout task-amenability
!mv model.pt taskamenability/taskpredictor/.


Cloning into 'wildtrack-iqa'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 69 (delta 21), reused 56 (delta 12), pack-reused 0
Unpacking objects: 100% (69/69), done.
/content/wildtrack-iqa
Branch 'task-amenability' set up to track remote branch 'task-amenability' from 'origin'.
Switched to a new branch 'task-amenability'


In [3]:
from zipfile import ZipFile
# extract zip file
with ZipFile('WildTrack_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [1]:
%cd "wildtrack-iqa"

import torch
import torchvision
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
import albumentations as A

with open('taskamenability/taskpredictor/class_mapping.json') as data:
    mappings = json.load(data)

class_mapping = {item['model_idx']: item['class_name'] for item in mappings}
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = torch.jit.load('taskamenability/taskpredictor/model.pt').to(device)
transforms = A.load('taskamenability/taskpredictor/transforms.json')

/content/wildtrack-iqa


In [2]:
from torch.utils import data
import csv
import random
import torch
import yaml
import cv2
import os

from time import process_time
from torch.utils.data import Dataset

root_dir = '/content/wildtrack-iqa'
data_dir = root_dir + '/RAW'

def generate_data_files(sample_percentage=1):

    image_reference_list = []

    subdirectories = list(os.walk(data_dir, topdown=False))[:-1]
    for subdir in subdirectories:
        image_location = subdir[0]
        images = subdir[2]
        species_rating = image_location.rsplit('/', 1)[-1].replace('_', ' ')
        score = int(species_rating.rsplit(' ', 1)[-1])
        species_class = species_rating.rsplit(' ', 1)[:-1][0]
        if len(species_class.rsplit(' ', 1)) > 1:
            species = species_class.rsplit(' ')[0]
            animal_class = ' '.join(species_class.rsplit(' ')[1:])
        else:
            animal_class = 'Unknown'
            species = species_class

        for image in images:
            image_reference = (image_location, species, animal_class, image, score)
            image_reference_list.append(image_reference)
        

    # shuffle then split
    seed = 1234
    random.Random(seed).shuffle(image_reference_list)

    train_index = int(len(image_reference_list) *0.6 *sample_percentage)
    test_index = -int(len(image_reference_list) * 0.2 * sample_percentage) 
    val_index = test_index * 2

    training = image_reference_list[:train_index]
    validation = image_reference_list[val_index:test_index]
    testing = image_reference_list[test_index:]
    return training, validation, testing

def convert_image(img_test):
    image = Image.open(img_test)
    image = image.resize((1024, 768))
    image = np.array(image)
    image = transforms(image=image)['image']
    x = torch.from_numpy(image).to(device)
    return x


def transform_images(images):
    x = np.array([convert_image(img[0]+'/'+img[3]) for img in images])
    y = np.array([0 if img[4] < 4 else 1 for img in images ])
    return x, y


In [3]:
# take a percentage of the full data set as a training/test/validation sample
sample_size=.01
train, valid, test = generate_data_files(sample_size)
x_train, y_train = transform_images(train)
x_val, y_val = transform_images(valid)
x_test, y_test = transform_images(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


**Execute the below installation only in Colab environment**

In [10]:
!pip install gym==0.17.3
!pip install stable-baselines==2.10.2
!pip install h5py==3.0.0
!pip install tensorflow==1.13.1
!pip install tensorboard==1.13.1
!pip install Keras==2.2.4
!pip install Keras-Applications==1.0.8
!pip install Keras-Preprocessing==1.1.2
!pip install keras-rl==0.4.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 33.4 MB/s 
     |████████████████████████████████| 1.0 MB 80.3 MB/s 
  Created wheel for gym: filename=gym-0.17.3-py3-none-any.whl size=1654653 sha256=07300996ad4112a543b6ba2b60452523275c8c6e2395c1ba183a5f3c58a6d653
  Stored in directory: /root/.cache/pip/wheels/d1/81/4b/dd9c029691022cb957398d1f015e66b75e37637dda61abdf58
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 240 kB 27.6 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 37.1 MB/s 
  Attempting uninstall: h5py
    Found existing ins

In [4]:
from taskamenability.interface import PPOInterface

img_shape = (416,555,3)
task_predictor = model
interface = PPOInterface(x_train, y_train, x_val, y_val, x_test, y_test, task_predictor, img_shape)


Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [5]:
interface.train(2)

Training started for 2 episodes:
test this out
Predicted Action: What Images Should be Used vs Not
[array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32), array([1.], dtype=float32)]


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:594.)
  return forward_call(*input, **kwargs)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2894.)
  return forward_call(*input, **kwargs)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: operator() sees varying value in profiling, ignoring and this should be handl

{'pred_classes': tensor([0]), 'scores': tensor([0.9985]), 'pred_boxes': tensor([[ 99.2085,  44.2886, 442.6307, 264.8299]])}
(1, array([1]))


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1130: UserWarning: operator() profile_node %248 : int = prim::profile_ivalue(%out_dtype.8)
 does not have profile information (Triggered internally at  ../torch/csrc/jit/codegen/cuda/graph_fuser.cpp:104.)
  return forward_call(*input, **kwargs)


{'pred_classes': tensor([0, 0]), 'scores': tensor([0.9713, 0.6345]), 'pred_boxes': tensor([[201.9142, 147.0838, 455.4426, 311.4117],
        [ 87.0187,  80.4855, 251.4566, 234.8643]])}
(1, array([1]))
{'pred_classes': tensor([0]), 'scores': tensor([0.9977]), 'pred_boxes': tensor([[135.0040,  15.6196, 520.8668, 303.3041]])}
(1, array([0]))
{'pred_classes': tensor([0]), 'scores': tensor([0.9535]), 'pred_boxes': tensor([[177.2644, 233.8972, 330.5865, 334.7251]])}
(1, array([0]))
{'pred_classes': tensor([0]), 'scores': tensor([0.9983]), 'pred_boxes': tensor([[ 62.8145,  58.9444, 447.6633, 357.7259]])}
(1, array([1]))
{'pred_classes': tensor([0]), 'scores': tensor([0.9763]), 'pred_boxes': tensor([[142.4867,  77.8004, 452.4843, 316.9387]])}
(1, array([0]))
{'pred_classes': tensor([0, 0, 0]), 'scores': tensor([0.9914, 0.2011, 0.0725]), 'pred_boxes': tensor([[193.5932,  89.5177, 467.7228, 292.7657],
        [166.5195,  83.6837, 338.7125, 266.8720],
        [ 61.8758,  74.1864, 411.3309, 287.80

In [ ]:
# how to extract quality mode from interface and use for prediction
# eventually this should be a pytorch model and we'll need to account